In some other languages,
one can not recover from an error,
or it is difficult to recover from an error,
so one tests input before doing something that could provoke the error.
This technique is called **Look Before You Leap** (LBYL)

For example, in a C program,
one must avoid trying to calculate the square root of a negative number.

In [1]:
numbers = '''
2
1
0
-1
-2
'''
with open('numbers', 'w') as f:
    f.write(numbers)

In [2]:
c_program_source_code = r'''
#include <stdlib.h>
#include <stdio.h>

int foo(int x)
{
    return 10 / x;
}

int main(int argc, char *argv[])
{
    int x;
    int y;
    
    while (scanf("%d", &x) == 1) {
        y = foo(x);
        printf("foo(%d) --> %d\n", x, y);
    }        
}
'''

with open('foo.c', 'w') as f:
    f.write(c_program_source_code)

!make foo

cc     foo.c   -o foo


In [3]:
!./foo <numbers

foo(2) --> 5
foo(1) --> 10
Floating point exception (core dumped)


In [4]:
def foo(x):
    return 10 // x

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10


ZeroDivisionError: integer division or modulo by zero

In [5]:
c_program_source_code = r'''
#include <stdlib.h>
#include <stdio.h>

int foo(int x)
{
    if (x == 0)
        return 0;
    else
        return 10 / x;
}

int main(int argc, char *argv[])
{
    int x;
    int y;
    
    while (scanf("%d", &x) == 1) {
        y = foo(x);
        printf("foo(%d) --> %d\n", x, y);
    }        
}
'''

with open('foo.c', 'w') as f:
    f.write(c_program_source_code)

!make foo

cc     foo.c   -o foo


In [6]:
!./foo <numbers

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


One can use the same LBYL technique in Python.

In [7]:
def foo(x):
    if x == 0:
        return 0
    else :
        return 10 // x

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


Another technique is to just try stuff,
and if it blows up, do something else.

This technique is called **Easier to Ask Forgiveness than Permission** (EAFP).

Python makes it very easy to do something else
when something blows up.

In [8]:
def foo(x):
    try:
        y = 10 // x
    except ZeroDivisionError:
        y = 0
    return y

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


For that simple example,
EAFP does not have much if any benefit over LBYL.
For that simple example,
there is not much benefit  in the size or readability of the code.
However, for more complicated problems,
EAFP lets one write much simpler and readable code.

For example,
determining if a string can be converted to a float is complicated,
so just trying to convert it to a float,
and if it blows up,
do something else.

Show how to convert to float with LBYL.

try regex code
    nan
    inf
    <sign><digits>.<digits><[eE][+-]<digits>
    
disallow nan and inf
    where is exponent limit for exponent?
    do regex, then convert,
    then test for inf
    1.79769e308
    
def mogrify(s):
    try:
        x = float(s)
    except ValueError:
        x = float('nan')
    return x

In [ ]:
def mogrify(types, s):
    '''Converts s to the first type in types that works.'''
    for type_ in types:
        try:
            x = type_(s)
        except:
            continue
        break
            
    return x

In [ ]:
types = (int, float, str)
input_strings = (
    '-1234',
    '-1234.',
    '-1234.nana',
    #'nan',
)

In [ ]:
for s in input_strings:
    x = mogrify(types, s)
    print('%r -> %r (%s)' % (s, x, x.__class__.__name__))

That works, but is bad.

One should not have naked excepts.

One should specify exactly which exceptions one is going to catch.
I often do that by just provoking the error
and seeing what it complains about
instead of RTFM.

Naked excepts are bad.

Often provoke the error with known bad input
just to see what error is provoked,
then paste that error into the except statement.

In [ ]:
int('-1234.')

In [ ]:
def mogrify(types, s):
    '''Converts s to the first type in types that works.'''
    for type_ in types:
        try:
            x = type_(s)
        except ValueError:
            continue
        break
            
    return x

In [ ]:
for s in input_strings:
    x = mogrify(types, s)
    print('%r -> %r (%s)' % (s, x, x.__class__.__name__))